# Busca com K-NN - LAB 02

Objetivo: A partir de palavras-chave encontrar as notícias mais similares usando busca K-NN (K-nearest neighbors) e procurar as notícias mais similares a uma notícia do corpus.

O K-NN tenta classificar cada amostra de um conjunto de dados avaliando sua distância em relação aos vizinhos mais próximos. Exemplo de funções de distância é a Distância Euclidiana, a Distância Manhattan, Similaridade de cossenos.

Nesse trabalho usa o pacote Scikit-Learn. 
Usa a similaridade de cossenos para o cálculo da distância e classificação Binária, Term Frequency (TF) e Term Frequency-Inverse Document Frequency (TF-IDF).


Pegando o dataframe:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
url = "https://gist.githubusercontent.com/issilva5/665cb1a2fee6d0e7a144dd1f48fd07d4/raw/ba4b864530493ab1ef8bd59f695d43cba726d3b8/noticias_folha_sample.csv"
df = pd.read_csv(url)
df.head(10)

,titulo,conteudo
0,Desenhos que icebergs deixam no fundo do mar c...,A imagem acima parece um desenho com giz de ce...
1,Bolas de neve gigantes aparecem em praia da Si...,"Moradores do Golfo de Ob, no noroeste da Sibér..."
2,Projeto de termelétrica no litoral de SP gera ...,O plano de construção de um complexo termelétr...
3,Cidades com mais mata atlântica estão no litor...,"Ubatuba, Ilhabela e São Sebastião, todas no li..."
4,Câmara aprova projeto que facilita exploração ...,Após mais de sete meses de impasse e intensas ...
5,"Oceanos perderam 2% de oxigênio desde 1960, ap...",Os oceanos do mundo perderam mais de 2% do seu...
6,Agricultura urbana gera renda e comida limpa n...,"José Aparecido Candido Vieira, 65, era vendedo..."
7,ONU e França pedem que Trump respeite acordo d...,O secretário-geral da Organização das Nações U...
8,Isenção de imposto no Paraná ajuda donos de im...,O intenso pio das corujas foi o que mais impre...
9,Temer revoga decreto sobre reserva mineral e a...,Menos de uma semana depois de anunciar a extin...


In [ ]:
df.shape # dimensões

(8294, 2)

## Buscando notícias usando palavras-chave

Definindo conjunto de palavras-chaves:

In [ ]:
keyword = "clima ecossistema temperatura mudança climática"
k=3

Funções que vou utilizar:

In [ ]:
# Criação do vetorizador binário e matriz
def buildCountVectorizer_binary(docs):
  vectorizer = CountVectorizer(binary=True)
  matrix = vectorizer.fit_transform(docs)
  return matrix, vectorizer

# Criação do vetorizador TF e matriz
def buildTFVectorizer(docs):
  vect = TfidfVectorizer(use_idf=False)
  matrix = vect.fit_transform(docs)
  return matrix, vect

# Criação do vetorizador TF-IDF e matriz
def buildTFIDFVectorizer(docs):
  vect = TfidfVectorizer()
  matrix = vect.fit_transform(docs)
  return matrix, vect

# Vetoriza a keyword
def vectorizerKeyword(keyword, vectorizer):
    query_name = [keyword]
    query = vectorizer.transform(query_name)
    return query

# Busca os k vizinhos mais proximo
def getNearestNeighbors(k, matrix, query):
  neigh = NearestNeighbors(n_neighbors=k)
  neigh.fit(matrix)
  knn = neigh.kneighbors(query,k)
  return knn

### Tipo de representação: Binário

#### Busca por título
Primeiro usando só o **titulo** para buscar as notícias mais similares:

In [ ]:
binary_matrix, vectorizer = buildCountVectorizer_binary(df['titulo'])

In [ ]:
# Vetorizar a nova sentença
query = vectorizerKeyword(keyword, vectorizer)

# Calcular a similaridade do cosseno entre a nova sentença e os dados existentes
cosine_similarities = cosine_similarity(query, binary_matrix).flatten()

# Obter os índices das k frases mais semelhantes
indices = cosine_similarities.argsort()[-k:][::-1]


In [ ]:
pd.options.display.max_colwidth = 150
result_binary_titulo = df.iloc[indices]['titulo']
result_binary_titulo

475                                                         Clima marombado
5598             Obama admite responsabilidade dos EUA em mudança climática
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object

#### Busca por título e conteúdo
Usando o **titulo** e **conteudo**, para isso criei uma coluna combinando as duas colunas.

In [ ]:
# Criar uma coluna combinando título e conteúdo
df['titulo_conteudo'] = df['titulo'] + ': ' + df['conteudo']
df.head(5)

,titulo,conteudo,titulo_conteudo
0,Desenhos que icebergs deixam no fundo do mar contam história do clima,"A imagem acima parece um desenho com giz de cera feito por uma criança. Mas, na verdade, trata-se de uma foto das imensas ""cicatrizes"" deixadas n...",Desenhos que icebergs deixam no fundo do mar contam história do clima: A imagem acima parece um desenho com giz de cera feito por uma criança. Ma...
1,Bolas de neve gigantes aparecem em praia da Sibéria,"Moradores do Golfo de Ob, no noroeste da Sibéria (Rússia), foram surpreendidos neste sábado com milhares de bolas de neve espalhadas pela praia. ...","Bolas de neve gigantes aparecem em praia da Sibéria: Moradores do Golfo de Ob, no noroeste da Sibéria (Rússia), foram surpreendidos neste sábado c..."
2,Projeto de termelétrica no litoral de SP gera preocupações ambientais,"O plano de construção de um complexo termelétrico em Peruíbe, na Baixada Santista, tem gerado controvérsia na cidade, alarmado biólogos e ambienta...","Projeto de termelétrica no litoral de SP gera preocupações ambientais: O plano de construção de um complexo termelétrico em Peruíbe, na Baixada Sa..."
3,"Cidades com mais mata atlântica estão no litoral norte de SP, diz estudo","Ubatuba, Ilhabela e São Sebastião, todas no litoral norte de São Paulo, são os municípios com mais mata atlântica no Estado. De acordo com o Atla...","Cidades com mais mata atlântica estão no litoral norte de SP, diz estudo: Ubatuba, Ilhabela e São Sebastião, todas no litoral norte de São Paulo, ..."
4,Câmara aprova projeto que facilita exploração de biodiversidade,"Após mais de sete meses de impasse e intensas negociações, a Câmara dos Deputados aprovou na noite desta segunda-feira (9) um projeto que facilita...","Câmara aprova projeto que facilita exploração de biodiversidade: Após mais de sete meses de impasse e intensas negociações, a Câmara dos Deputados..."


In [ ]:
binary_matrix_tc, vectorizer_tc = buildCountVectorizer_binary(df['titulo_conteudo'])

In [ ]:
query = vectorizerKeyword(keyword, vectorizer_tc)

cosine_similarities = cosine_similarity(query, binary_matrix_tc).flatten()
indices_tc = cosine_similarities.argsort()[-k:][::-1]

In [ ]:
result_binary_tc = df.iloc[indices_tc]['titulo']
result_binary_tc

1468           Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
544     Países reunidos em NY querem barrar uso de HFC, um dos gases do efeito estufa
5661         ONGs pressionam por mais proteção a países ameaçados pelo clima na COP21
Name: titulo, dtype: object

### Tipo de representação: TF

#### Busca por título
Primeiro usando só o **titulo** para buscar as notícias mais similares:

In [ ]:
matrix_tf, vect_tf = buildTFVectorizer(df["titulo"])

In [ ]:
query = vectorizerKeyword(keyword, vect_tf)
knn_tf  = getNearestNeighbors(k, matrix_tf, query)

In [ ]:
pd.options.display.max_colwidth = 150
result_tf_titulo = df.iloc[knn_tf[1][0]]['titulo']
result_tf_titulo

475                                                         Clima marombado
5598             Obama admite responsabilidade dos EUA em mudança climática
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object

#### Busca por título e conteúdo
Agora usando o **titulo** e **conteudo**:

In [ ]:
matrix_tf_tc, vect_tf_tc = buildTFVectorizer(df["titulo_conteudo"])

In [ ]:
query = vectorizerKeyword(keyword, vect_tf_tc)
knn_tf = getNearestNeighbors(k, matrix_tf_tc, query)

In [ ]:
result_tf_tc = df.iloc[knn_tf[1][0]]['titulo']
result_tf_tc

1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
6313                                          Ganhando o jogo das nossas vidas
475                                                            Clima marombado
Name: titulo, dtype: object

### Tipo de representação: TF-IDF

#### Busca por título
Primeiro usando só o **titulo** para buscar as notícias mais similares:

In [ ]:
matrix_tfidf, vect_tfidf = buildTFIDFVectorizer(df["titulo"])

In [ ]:
query = vectorizerKeyword(keyword, vect_tfidf)
knn_tfidf  = getNearestNeighbors(k, matrix_tfidf, query)

In [ ]:
result_tfidf_titulo = df.iloc[knn_tfidf[1][0]]['titulo']
result_tfidf_titulo

5598                Obama admite responsabilidade dos EUA em mudança climática
5960       EUA têm consciência sobre mudança climática, diz Obama na Argentina
1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
Name: titulo, dtype: object

#### Busca por título e conteúdo
Agora usando o **titulo** e **conteudo**:

In [ ]:
matrix_tfidf_tc, vect_tfidf_tc = buildTFIDFVectorizer(df['titulo_conteudo'])

In [ ]:
query = vectorizerKeyword(keyword, vect_tfidf_tc)
knn_tfidf  = getNearestNeighbors(k, matrix_tfidf_tc,query)

In [ ]:
result_tfidf_tc = df.iloc[knn_tfidf[1][0]]['titulo']
result_tfidf_tc

6313                                          Ganhando o jogo das nossas vidas
1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
5516           Limbo jurídico aguarda milhões de futuros "refugiados do clima"
Name: titulo, dtype: object

Podemos ver que em primeiro ficou um titulo que
não parece que tem muita semelhança com as palavras chaves, mas o contéudo da reportagem fala sobre as mudanças climáticas e aumento de temperatura no planeta.

## Buscando notícias semelhantes


Selecionando uma notícia de indice 5598 com titulo 'Obama admite responsabilidade dos EUA em mudança climática':

In [ ]:
idx = 5598

### Tipo de representação: Binário

Fazendo a busca pelo **título**. Vou usar a mesma matriz binária usada na seção anterior: `binary_matrix`

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = binary_matrix.getrow(idx)

# Calcular a similaridade do cosseno entre a nova sentença e os dados existentes
cosine_similarities = cosine_similarity(query_doc, binary_matrix).flatten()

# Obter os índices das k frases mais semelhantes
# precisa retirar o ultimo indice que seria da propria reportagem (por isso [-k-1:-1] )
indices = cosine_similarities.argsort()[-k-1:-1][::-1]

result_noticia_binary_titulo = df.iloc[indices]['titulo']
result_noticia_binary_titulo

5960             EUA têm consciência sobre mudança climática, diz Obama na Argentina
5968                           Mudança na demografia dos EUA enfraquece republicanos
6130    Declínio dos EUA é 'ficção', diz Obama em último discurso do Estado da União
Name: titulo, dtype: object

Agora utilizando o **título** e **conteúdo** para a busca vou utilizar a mesma matriz binária usada na seção anterior: `binary_matrix_tc`

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = binary_matrix_tc.getrow(idx)

# Calcular a similaridade do cosseno entre a nova sentença e os dados existentes
cosine_similarities = cosine_similarity(query_doc, binary_matrix_tc).flatten()

# Obter os índices das k frases mais semelhantes
indices_tc = cosine_similarities.argsort()[-k-1:-1][::-1]

result_noticia_binary_tc = df.iloc[indices_tc]['titulo']
result_noticia_binary_tc

6081    Trump elogiou Duterte por guerra às drogas nas Filipinas, diz jornal
5960     EUA têm consciência sobre mudança climática, diz Obama na Argentina
5318                 Agenda de Dilma e Merkel para o clima carece de ambição
Name: titulo, dtype: object

### Tipo de representação: TF

Vou usar a mesma matriz TF usada na seção anterior: `matrix_tf`

Obs.: estou tirando a própria notícia no resultado (por isso [1:])

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = matrix_tf.getrow(idx)

knn_tf  = getNearestNeighbors(k+1, matrix_tf, query_doc)

result_noticia_tf_titulo = df.iloc[knn_tf[1][0]]['titulo'][1:]
result_noticia_tf_titulo

5960       EUA têm consciência sobre mudança climática, diz Obama na Argentina
5549    Obama vai criar primeiro monumento a direitos dos homossexuais dos EUA
5968                     Mudança na demografia dos EUA enfraquece republicanos
Name: titulo, dtype: object

Agora utilizando o **título** e **conteúdo** para a busca vou utilizar a mesma matriz TF usada na seção anterior: `matrix_tf_tc`

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = matrix_tf_tc.getrow(idx)

knn_tf  = getNearestNeighbors(k+1, matrix_tf_tc, query_doc)

result_noticia_tf_tc = df.iloc[knn_tf[1][0]]['titulo'][1:]
result_noticia_tf_tc

6117    Discurso otimista de Obama difere de visão republicana sobre os EUA
13       Vitória de Trump provoca silêncio e receio em conferência do clima
5516        Limbo jurídico aguarda milhões de futuros "refugiados do clima"
Name: titulo, dtype: object

### Tipo de representação: TF-IDF

Vou usar a mesma matriz TF-IDF usada na seção anterior: `matrix_tfidf`

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = matrix_tfidf.getrow(idx)

knn_tfidf  = getNearestNeighbors(k+1, matrix_tfidf, query_doc)

result_noticia_tfidf_titulo = df.iloc[knn_tfidf[1][0]]['titulo'][1:]
result_noticia_tfidf_titulo

5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
6295                                   A responsabilidade fiscal do governo
6327                                        Misericórdia e responsabilidade
Name: titulo, dtype: object

Agora utilizando o **título** e **conteúdo** para a busca vou utilizar a mesma matriz TF-IDF usada na seção anterior: `matrix_tfidf_tc`

In [ ]:
# Pegando o titulo de entrada para a busca
query_doc = matrix_tfidf_tc.getrow(idx)

knn_tfidf  = getNearestNeighbors(k+1, matrix_tfidf_tc, query_doc)

result_noticia_tfidf_tc = df.iloc[knn_tfidf[1][0]]['titulo'][1:]
result_noticia_tfidf_tc

13       Vitória de Trump provoca silêncio e receio em conferência do clima
6117    Discurso otimista de Obama difere de visão republicana sobre os EUA
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object

# Análise dos resultados

Palavras-chave usada: **clima ecossistema temperatura mudança climática**

Buscando com palavras-chaves: matriz binária de representação usando busca apenas no título

In [ ]:
result_binary_titulo

475                                                         Clima marombado
5598             Obama admite responsabilidade dos EUA em mudança climática
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object

Buscando com palavras-chave: matriz binária de representação usando busca no título e conteúdo


In [ ]:
result_binary_tc

1468           Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
544     Países reunidos em NY querem barrar uso de HFC, um dos gases do efeito estufa
5661         ONGs pressionam por mais proteção a países ameaçados pelo clima na COP21
Name: titulo, dtype: object

Buscando com palavras-chave: matriz TF de representação usando busca apenas no título


In [ ]:
result_tf_titulo

475                                                         Clima marombado
5598             Obama admite responsabilidade dos EUA em mudança climática
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object

Buscando com palavras-chave: matriz TF de representação usando busca no título e conteúdo


In [ ]:
result_tf_tc

1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
6313                                          Ganhando o jogo das nossas vidas
475                                                            Clima marombado
Name: titulo, dtype: object

Buscando com palavras-chave: matriz TF-IDF de representação usando busca apenas no título


In [ ]:
result_tfidf_titulo

5598                Obama admite responsabilidade dos EUA em mudança climática
5960       EUA têm consciência sobre mudança climática, diz Obama na Argentina
1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
Name: titulo, dtype: object

Buscando com palavras-chave: matriz TF-IDF de representação usando busca no título e conteúdo


In [ ]:
result_tfidf_tc

6313                                          Ganhando o jogo das nossas vidas
1468    Temperatura sobe e São Paulo terá 'clima de verão' a partir do feriado
5516           Limbo jurídico aguarda milhões de futuros "refugiados do clima"
Name: titulo, dtype: object



> Podemos perceber que alguns resultados são iguais, porem ranqueadas diferente, mas todos retornam resultados que contem as palavras-chave. Usando TF com a busca em título e conteúdo podemos ver um resultado que parece não ter nada em comum com as palavras-chave: "Ganhando o jogo das nossas vidas", mas o conteúdo da notícia tem bastante similaridade, então podemos concluir que usando a busca também no conteúdo conseguimos resultados melhores. Apesar dos resultados parecidos, o TF-IDF se saiu melhor, pois quantifica a raridade das palavras, olhando para o conteúdo dos 3 títulos que retornou eles tem muito mais similaridade com as palavras-chave (“clima ecossistema temperatura mudança climática”)



A outra notícia escolhida foi: **"Obama admite responsabilidade dos EUA em mudança climática"**

Buscando com outra notícia: matriz binária de representação usando busca apenas no título


In [ ]:
result_noticia_binary_titulo

5960             EUA têm consciência sobre mudança climática, diz Obama na Argentina
5968                           Mudança na demografia dos EUA enfraquece republicanos
6130    Declínio dos EUA é 'ficção', diz Obama em último discurso do Estado da União
Name: titulo, dtype: object

Buscando com outra notícia: matriz binária de representação usando busca no título e conteúdo


In [ ]:
result_noticia_binary_tc

6081    Trump elogiou Duterte por guerra às drogas nas Filipinas, diz jornal
5960     EUA têm consciência sobre mudança climática, diz Obama na Argentina
5318                 Agenda de Dilma e Merkel para o clima carece de ambição
Name: titulo, dtype: object

Buscando com outra notícia: matriz TF de representação usando busca apenas no título


In [ ]:
result_noticia_tf_titulo

5960       EUA têm consciência sobre mudança climática, diz Obama na Argentina
5549    Obama vai criar primeiro monumento a direitos dos homossexuais dos EUA
5968                     Mudança na demografia dos EUA enfraquece republicanos
Name: titulo, dtype: object

Buscando com outra notícia: matriz TF de representação usando busca no título e conteúdo


In [ ]:
result_noticia_tf_tc

6117    Discurso otimista de Obama difere de visão republicana sobre os EUA
13       Vitória de Trump provoca silêncio e receio em conferência do clima
5516        Limbo jurídico aguarda milhões de futuros "refugiados do clima"
Name: titulo, dtype: object

Buscando com outra notícia: matriz TF-IDF de representação usando busca apenas no título


In [ ]:
result_noticia_tfidf_titulo

5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
6295                                   A responsabilidade fiscal do governo
6327                                        Misericórdia e responsabilidade
Name: titulo, dtype: object

Buscando com outra notícia: matriz TF-IDF de representação usando busca no título e conteúdo


In [ ]:
result_noticia_tfidf_tc

13       Vitória de Trump provoca silêncio e receio em conferência do clima
6117    Discurso otimista de Obama difere de visão republicana sobre os EUA
5960    EUA têm consciência sobre mudança climática, diz Obama na Argentina
Name: titulo, dtype: object



> Usando outra notícia como busca, os resultados foram mais variados do que usando palavras-chave. O melhor resultado foi usando TF-IDF com busca com título e conteúdo que as 3 notícias que resultou tinham mais similaridade do que as outras, isso foi possível por que o TF-IDF leva em conta a raridade das palavras em todos os documentos e a importância das palavras no documento que foi usado como busca.

